In [1]:
import sys
import json
import numpy as np

from src.env import StockTradingEnv
from src.agent import DDPG_Hedger
from src.network import MLP

/Users/mac/Desktop/Repos/ml_project_2_deep_hedging/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("model/hypparams.json", "r") as file:
    hyp_params = json.load(file)


In [30]:
import torch
import torch.nn as nn
import torch.optim as optim

from copy import deepcopy
from src.buffer import ExpReplay
from collections import namedtuple
from torch.distributions import Normal


class DDPG_Hedger:
    def __init__(
        self,
        Actor: nn.Module,
        Critic_1: nn.Module,
        Critic_2: nn.Module,
        actor_lr: float,
        critic_lr: float,
        disc_rate: float = 1,
        batch_size: int = 32,
    ):

        # params
        self.gamma = disc_rate
        self.tau = 0.01
        self.batch_size = batch_size

        # experience replay related
        self.transition = namedtuple(
            "Transition",
            ("state", "action", "reward", "next_state", "done"),
        )
        self.buffer = ExpReplay(10000, self.transition)

        # define actor and critic ANN.
        self.actor = Actor
        self.critic_1 = Critic_1  # mean(cost)
        self.critic_2 = Critic_2  # std(cost)

        # loss function for critic
        self.critic_loss = nn.MSELoss()

        # define optimizer for Actor and Critic network
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=actor_lr)
        self.critic_1_optimizer = optim.Adam(self.critic_1.parameters(), lr=critic_lr)
        self.critic_2_optimizer = optim.Adam(self.critic_2.parameters(), lr=critic_lr)

        # define target network needed for DDPG optimization
        self.actor_target = deepcopy(self.actor)
        self.critic_1_target = deepcopy(self.critic_1)
        self.critic_2_target = deepcopy(self.critic_2)

    def reset(self):
        self.buffer.clear()

    def store(self, *args):
        self.buffer.store(*args)

    def act(self, state: list, sigma: float = 0.2):
        """
        We use policy function to find the deterministic action instead of distributions
        which is parametrized by distribution parameters learned from the policy.

        Here, state input prompts policy network to output a single or multiple-dim
        actions.
        :param state:
        :return:
        """
        x = torch.tensor(state).to(torch.float64)
        action = self.actor.forward(x)
        noise = torch.normal(mean=torch.Tensor([0]),std=torch.Tensor([sigma]))
        print(action, noise)
        return (
            torch.clip((action - 0.5) * 2 + noise, -state[0], 1.0 - state[0])
            .detach()
            .numpy()
        )

    def update(self, output=False):
        # calculate return of all times in the episode
        if self.buffer.len() < self.batch_size:
            return

        transitions = self.buffer.sample(self.batch_size)
        batch = self.transition(*zip(*transitions))

        # extract variables from sampled batch.
        states = torch.tensor(batch.state)
        actions = torch.tensor(batch.action)
        rewards = torch.tensor(batch.reward)
        dones = torch.tensor(batch.done).float()
        next_states = torch.tensor(batch.next_state)

        # compute Q_1 loss
        Q_1 = self.critic_1(torch.hstack([states, actions]))
        y_1 = rewards + self.gamma * (1 - dones) * self.critic_1_target(
            torch.hstack([next_states, self.actor_target(next_states)]).detach()
        )

        critic_loss_1 = self.critic_loss(Q_1, y_1)
        
        # Optimize the critic Q_1
        self.critic_1_optimizer.zero_grad()
        critic_loss_1.backward()
        self.critic_1_optimizer.step()

        # compute Q_2 loss
        Q_2 = self.critic_2(torch.hstack([states, actions]))
        y_2 = (
            rewards**2
            + (self.gamma**2)
            * (1 - dones)
            * self.critic_2_target(
                torch.hstack([next_states, self.actor_target(next_states)]).detach()
            )
            + 2
            * self.gamma
            * rewards
            * self.critic_1_target(
                torch.hstack([next_states, self.actor_target(next_states)]).detach()
            )
        )

        critic_loss_2 = self.critic_loss(Q_2, y_2)
        
        # Optimize the critic Q_2
        self.critic_2_optimizer.zero_grad()
        critic_loss_2.backward()
        self.critic_2_optimizer.step()


        # Get actor loss
        state_action = torch.hstack([states, self.actor(states)])
        cost_variance = (
            self.critic_2(state_action)
            - self.critic_1(state_action) ** 2
        )
        #print(self.critic_1(state_action)[:3], self.critic_2(state_action)[:3])
        actor_loss = (
            self.critic_1(state_action)
            + 1.5 * torch.sqrt(torch.where(cost_variance < 0, 0, cost_variance))
        ).mean()

        # Optimize the actor
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        print(critic_loss_1, critic_loss_2, actor_loss)

        if output:
            return actor_loss.detach().item()

    def polyak_update(self):
        # Update the frozen target models
        for trg_param, src_param in zip(
            list(self.critic_1_target.parameters()), list(self.critic_1.parameters())
        ):
            trg_param = trg_param * (1.0 - self.tau) + src_param * self.tau

        # Update the frozen target models
        for trg_param, src_param in zip(
            list(self.critic_2_target.parameters()), list(self.critic_2.parameters())
        ):
            trg_param = trg_param * (1.0 - self.tau) + src_param * self.tau


        for trg_param, src_param in zip(
            list(self.actor_target.parameters()), list(self.actor.parameters())
        ):
            trg_param = trg_param * (1.0 - self.tau) + src_param * self.tau

    def save(self, filename):
        torch.save(self.critic.state_dict(), filename + "_critic")
        torch.save(self.critic_optimizer.state_dict(), filename + "_critic_hyp_params")

        torch.save(self.actor.state_dict(), filename + "_actor")
        torch.save(self.actor_optimizer.state_dict(), filename + "_actor_hyp_params")

    def load(self, filename):
        self.critic.load_state_dict(torch.load(filename + "_critic"))
        self.critic_optimizer.load_state_dict(
            torch.load(filename + "_critic_hyp_params")
        )

        self.actor.load_state_dict(torch.load(filename + "_actor"))
        self.actor_optimizer.load_state_dict(torch.load(filename + "_actor_hyp_params"))

        # define target network needed for DDPG optimization
        self.actor_target = deepcopy(self.actor)
        self.critic_target = deepcopy(self.critic)

In [48]:
BATCH_SIZE = 16
N_EPISODE = 1

env = StockTradingEnv(reset_path=True)

nHidden = hyp_params["hidden_dim"]
actor_lr = 10 ** hyp_params["actor_lr"]
critic_lr = 10 ** hyp_params["critic_lr"]
trg_update = hyp_params["polyak_update_freq"]
nState, nAction = env.observation_space.shape[0], env.action_space.shape[0]  # 3, 1

actor_lr

actor = MLP(nState, nHidden, nAction, "Sigmoid")
qnet_1 = MLP(nState + nAction, nHidden, nAction, "")
qnet_2 = MLP(nState + nAction, nHidden, nAction, "")
agent = DDPG_Hedger(actor, qnet_1, qnet_2, actor_lr, critic_lr, 1, BATCH_SIZE)

In [49]:
target_rewards = []
noise_std = 1

for episode in range(N_EPISODE):
    # reset state
    state = env.reset()  # s_0
    ep_tot_reward = 0

    if episode > N_EPISODE - 30:
        noise_std = 0.0001

    i = 0
    while True:
        # take action given state
        print(noise_std)
        print(f'------- step {i+1} action:')
        action = agent.act(state, noise_std)
        
        # take next step of the environment
        next_state, reward, done = env.step(action)

        # record interaction between environment and the agent
        agent.store(state, action, reward, next_state, done)

        ep_tot_reward -= reward
        state = next_state
        
        agent.update()

        i +=1 
        if done:
            break
        
    print(f"Episode {episode} Reward: {ep_tot_reward}")
    # store total rewards after some training is done
    # we only consider alst 10 total rewards as a quantity to minimize
    if episode > N_EPISODE - 30:
        target_rewards.append(ep_tot_reward)

    if episode % trg_update == 0:  # update target network
        agent.polyak_update()

#print(np.mean(target_rewards))


1
------- step 1 action:
tensor([0.3838], grad_fn=<SigmoidBackward0>) tensor([-0.5296])
1
------- step 2 action:
tensor([0.3839], grad_fn=<SigmoidBackward0>) tensor([0.6695])
1
------- step 3 action:
tensor([0.3845], grad_fn=<SigmoidBackward0>) tensor([-0.4908])
1
------- step 4 action:
tensor([0.3840], grad_fn=<SigmoidBackward0>) tensor([0.1980])
1
------- step 5 action:
tensor([0.3841], grad_fn=<SigmoidBackward0>) tensor([0.4619])
1
------- step 6 action:
tensor([0.3845], grad_fn=<SigmoidBackward0>) tensor([0.9056])
1
------- step 7 action:
tensor([0.3855], grad_fn=<SigmoidBackward0>) tensor([0.3452])
1
------- step 8 action:
tensor([0.3857], grad_fn=<SigmoidBackward0>) tensor([-0.0576])
1
------- step 9 action:
tensor([0.3856], grad_fn=<SigmoidBackward0>) tensor([-1.1464])
1
------- step 10 action:
tensor([0.3849], grad_fn=<SigmoidBackward0>) tensor([-1.9654])
1
------- step 11 action:
tensor([0.3851], grad_fn=<SigmoidBackward0>) tensor([0.9580])
1
------- step 12 action:
tensor([0.

In [26]:
torch.normal(mean=torch.Tensor([0]),std=torch.Tensor([1]))

tensor([-0.6170])

In [45]:
sigma = 1
torch.normal(mean=torch.Tensor([0]),std=torch.Tensor([sigma]))

tensor([0.5899])

In [35]:

# check the init params of actor and two critics
print('Actor')
for param in agent.actor.parameters():
    print(param)
    
print('Critic_1')
for param in agent.critic_1.parameters():
    print(param)
    
print('Critic_2')
for param in agent.critic_2.parameters():
    print(param)

Actor
Parameter containing:
tensor([[-0.3552, -0.4805,  0.3367],
        [-0.0299, -0.0076, -0.2759],
        [ 0.2599, -0.5416, -0.5670],
        [-0.5094, -0.0040,  0.6235],
        [-0.1893, -0.0390, -0.2158],
        [-0.5011,  0.5183,  0.6193],
        [ 0.5510, -0.0094, -0.5343],
        [-0.2291, -0.1014,  0.2488],
        [ 0.1784,  0.5880, -0.1074],
        [-0.4285,  0.2169, -0.6727],
        [ 0.3106,  0.4529,  0.3806],
        [ 0.2937,  0.4808, -0.2007],
        [ 0.4296, -0.2141,  0.0259]], requires_grad=True)
Parameter containing:
tensor([ 0.3141, -0.4637,  0.4545, -0.1267, -0.2552,  0.0329,  0.2607, -0.4837,
        -0.3835, -0.0641, -0.3224,  0.1753,  0.2736], requires_grad=True)
Parameter containing:
tensor([0.7990, 1.0038, 0.8314, 0.8326, 1.2060, 1.1693, 0.8005, 1.1530, 1.2068,
        0.8516, 1.0226, 0.8343, 0.7625], requires_grad=True)
Parameter containing:
tensor([ 0.2011, -0.0007,  0.1688,  0.1939, -0.2056,  0.1698,  0.1999,  0.0382,
         0.1996,  0.1364,  0.